# LOAD DATA

In [9]:
import argparse
import sys
import os
sys.path.append(os.getcwd())
from mtcnn.implement.imagedb import ImageDB

annotation_file = './anno_store/imglist_anno_12.txt'
model_store_path = './model_store'
end_epoch = 5
frequent = 200
lr = 0.01
batch_size = 512
use_cuda = False
imagedb = ImageDB(annotation_file)
gt_imdb = imagedb.load_imdb()
gt_imdb = imagedb.append_flipped_images(gt_imdb)

append flipped images to imdb 1957085


# Training

In [172]:
from mtcnn.implement.image_reader import TrainImageReader
import datetime
import os
from mtcnn.implement.models import PNet,RNet,ONet,LossFn
import torch
from torch.autograd import Variable
import mtcnn.implement.image_tools as image_tools
import numpy as np

def box_accuracy(g_box,o_box):
    return 1-np.sqrt(np.abs(np.product(g_box-o_box))/np.product(o_box))
def get_image_box_accuracy(img,oboxs,mtcnn_detector):
    bboxs, landmarks = mtcnn_detector.detect_face(img)
    if len(bboxs)==0:
        return 0
    return box_accuracy(bboxs[0][:4],oboxs)
def compute_accuracy(prob_cls, gt_cls):

    prob_cls = torch.squeeze(prob_cls)
    gt_cls = torch.squeeze(gt_cls)

    #we only need the detection which >= 0
    mask = torch.ge(gt_cls,0)
    #get valid element
    valid_gt_cls = torch.masked_select(gt_cls,mask)
    valid_prob_cls = torch.masked_select(prob_cls,mask)
    size = min(valid_gt_cls.size()[0], valid_prob_cls.size()[0])
    prob_ones = torch.ge(valid_prob_cls,0.6).float()
    right_ones = torch.eq(prob_ones,valid_gt_cls).float()

    ## if size == 0 meaning that your gt_labels are all negative, landmark or part

    return torch.div(torch.mul(torch.sum(right_ones),float(1.0)),float(size))  ## divided by zero meaning that your gt_labels are all negative, landmark or part


def train_pnet(model_store_path, end_epoch,imdb,
              batch_size,frequent=10,base_lr=0.01,use_cuda=False):
    if not os.path.exists(model_store_path):
        os.makedirs(model_store_path)

    lossfn = LossFn()
    net = PNet(is_train=True, use_cuda=use_cuda)
    net.train()

    if use_cuda:
        net.cuda()
    optimizer = torch.optim.Adam(net.parameters(), lr=base_lr)
    train_data=TrainImageReader(imdb,12,batch_size,shuffle=True)
    frequent = 100
    end_epoch=10
    for cur_epoch in range(1,end_epoch+1):
        #print("Hi")
        
        train_data.reset() # shuffle
        #print(enumerate(train_data))
        for batch_idx,(image,(gt_label,gt_bbox,gt_landmark))in enumerate(train_data):
            im_tensor = [ image_tools.convert_image_to_tensor(image[i,:,:,:]) for i in range(image.shape[0]) ]
            im_tensor = torch.stack(im_tensor)

            im_tensor = Variable(im_tensor)
            gt_label = Variable(torch.from_numpy(gt_label).float())

            gt_bbox = Variable(torch.from_numpy(gt_bbox).float())
            # gt_landmark = Variable(torch.from_numpy(gt_landmark).float())

            if use_cuda:
                im_tensor = im_tensor.cuda()
                gt_label = gt_label.cuda()
                gt_bbox = gt_bbox.cuda()
                # gt_landmark = gt_landmark.cuda()

            cls_pred, box_offset_pred = net(im_tensor)
            # all_loss, cls_loss, offset_loss = lossfn.loss(gt_label=label_y,gt_offset=bbox_y, pred_label=cls_pred, pred_offset=box_offset_pred)

            cls_loss = lossfn.cls_loss(gt_label,cls_pred)
            box_offset_loss = lossfn.box_loss(gt_label,gt_bbox,box_offset_pred)
            # landmark_loss = lossfn.landmark_loss(gt_label,gt_landmark,landmark_offset_pred)

            all_loss = cls_loss*1.0+box_offset_loss*0.5

            if batch_idx %frequent==0:
                accuracy=compute_accuracy(cls_pred,gt_label)

                show1 = accuracy.data.cpu().numpy()
                show2 = cls_loss.data.cpu().numpy()
                show3 = box_offset_loss.data.cpu().numpy()
                # show4 = landmark_loss.data.cpu().numpy()
                show5 = all_loss.data.cpu().numpy()

                print("%s : Epoch: %d, Step: %d, accuracy: %s, det loss: %s, bbox loss: %s, all_loss: %s, lr:%s "%(datetime.datetime.now(),cur_epoch,batch_idx, show1,show2,show3,show5,base_lr))

            optimizer.zero_grad()
            all_loss.backward()
            optimizer.step()

        torch.save(net.state_dict(), os.path.join(model_store_path,"pnet_epoch_%d.pt" % cur_epoch))
        torch.save(net, os.path.join(model_store_path,"pnet_epoch_model_%d.pkl" % cur_epoch))




def train_rnet(model_store_path, end_epoch,imdb,
              batch_size,frequent=50,base_lr=0.01,use_cuda=True):
    if not os.path.exists(model_store_path):
        os.makedirs(model_store_path)

    lossfn = LossFn()
    net = RNet(is_train=True, use_cuda=use_cuda)
    net.train()
    if use_cuda:
        net.cuda()

    optimizer = torch.optim.Adam(net.parameters(), lr=base_lr)

    train_data=TrainImageReader(imdb,128,batch_size,shuffle=True)


    for cur_epoch in range(1,end_epoch+1):
        train_data.reset()

        for batch_idx,(image,(gt_label,gt_bbox,gt_landmark))in enumerate(train_data):

            im_tensor = [ image_tools.convert_image_to_tensor(image[i,:,:,:]) for i in range(image.shape[0]) ]
            im_tensor = torch.stack(im_tensor)

            im_tensor = Variable(im_tensor)
            gt_label = Variable(torch.from_numpy(gt_label).float())

            gt_bbox = Variable(torch.from_numpy(gt_bbox).float())
            gt_landmark = Variable(torch.from_numpy(gt_landmark).float())

            if use_cuda:
                im_tensor = im_tensor.cuda()
                gt_label = gt_label.cuda()
                gt_bbox = gt_bbox.cuda()
                gt_landmark = gt_landmark.cuda()

            cls_pred, box_offset_pred = net(im_tensor)
            # all_loss, cls_loss, offset_loss = lossfn.loss(gt_label=label_y,gt_offset=bbox_y, pred_label=cls_pred, pred_offset=box_offset_pred)
            cls_loss = lossfn.cls_loss(gt_label,cls_pred)
            box_offset_loss = lossfn.box_loss(gt_label,gt_bbox,box_offset_pred)
            # landmark_loss = lossfn.landmark_loss(gt_label,gt_landmark,landmark_offset_pred)
            all_loss = cls_loss*1.0+box_offset_loss*0.5

            if batch_idx%frequent==0:
                accuracy=compute_accuracy(cls_pred,gt_label)

                show1 = accuracy.data.cpu().numpy()
                show2 = cls_loss.data.cpu().numpy()


                print(" Epoch: %d, Step: %d, accuracy: %s, det loss: %s"%( cur_epoch, batch_idx, show1, show2))

            optimizer.zero_grad()
            all_loss.backward()
            optimizer.step()
            if batch_idx>8000:
                break

        torch.save(net.state_dict(), os.path.join(model_store_path,"rnet_epoch_%d.pt" % cur_epoch))
        torch.save(net, os.path.join(model_store_path,"rnet_epoch_model_%d.pkl" % cur_epoch))


def train_onet(model_store_path, end_epoch,imdb,
              batch_size,frequent=50,base_lr=0.01,use_cuda=True):

    if not os.path.exists(model_store_path):
        os.makedirs(model_store_path)
    
    lossfn = LossFn()
    net = ONet(is_train=True)
    net.train()
    print(use_cuda)
    if use_cuda:
        net.cuda()

    optimizer = torch.optim.Adam(net.parameters(), lr=base_lr)

    train_data=TrainImageReader(imdb,48,batch_size,shuffle=True)
    
    for cur_epoch in range(1,end_epoch+1):

        train_data.reset()

        for batch_idx,(image,(gt_label,gt_bbox,gt_landmark))in enumerate(train_data):
            im_tensor = [ image_tools.convert_image_to_tensor(image[i,:,:,:]) for i in range(image.shape[0]) ]
            im_tensor = torch.stack(im_tensor)

            im_tensor = Variable(im_tensor)
            gt_label = Variable(torch.from_numpy(gt_label).float())

            gt_bbox = Variable(torch.from_numpy(gt_bbox).float())
            gt_landmark = Variable(torch.from_numpy(gt_landmark).float())

            if use_cuda:
                im_tensor = im_tensor.cuda()
                gt_label = gt_label.cuda()
                gt_bbox = gt_bbox.cuda()
                gt_landmark = gt_landmark.cuda()

            cls_pred, box_offset_pred, landmark_offset_pred = net(im_tensor)

            # all_loss, cls_loss, offset_loss = lossfn.loss(gt_label=label_y,gt_offset=bbox_y, pred_label=cls_pred, pred_offset=box_offset_pred)

            cls_loss = lossfn.cls_loss(gt_label,cls_pred)
            box_offset_loss = lossfn.box_loss(gt_label,gt_bbox,box_offset_pred)
            landmark_loss = lossfn.landmark_loss(gt_label,gt_landmark,landmark_offset_pred)

            all_loss = cls_loss*0.8+box_offset_loss*0.6+landmark_loss*1.5
            ##
            if batch_idx%frequent==0:
                accuracy=compute_accuracy(cls_pred,gt_label)
                show1 = accuracy.data.cpu().numpy()
                show2 = cls_loss.data.cpu().numpy()
                show3 = box_offset_loss.data.cpu().numpy()
                show4 = landmark_loss.data.cpu().numpy()
                show5 = all_loss.data.cpu().numpy()

                print("%s : Epoch: %d, Step: %d, accuracy: %s, det loss: %s, bbox loss: %s, landmark loss: %s, all_loss: %s, lr:%s "%(datetime.datetime.now(),cur_epoch,batch_idx, show1,show2,show3,show4,show5,base_lr))

            optimizer.zero_grad()
            all_loss.backward()
            optimizer.step()

        torch.save(net.state_dict(), os.path.join(model_store_path,"onet_epoch_%d.pt" % cur_epoch))
        torch.save(net, os.path.join(model_store_path,"onet_epoch_model_%d.pkl" % cur_epoch))


# Train PNet

In [15]:
end_epoch=5
def train_net(imagedb,gt_imdb,model_store_path,
                end_epoch=5, frequent=200, lr=0.01, batch_size=128, use_cuda=False):

    train_pnet(model_store_path=model_store_path, end_epoch=end_epoch, imdb=gt_imdb, batch_size=batch_size, frequent=frequent, base_lr=lr)
train_net(imagedb,gt_imdb, model_store_path,
            end_epoch, frequent, lr, batch_size, use_cuda)


2022-11-30 05:01:12.634316 : Epoch: 1, Step: 0, accuracy: 0.6210191, det loss: 0.7186119, bbox loss: 0.05406532, all_loss: 0.74564457, lr:0.01 
2022-11-30 05:01:39.571345 : Epoch: 1, Step: 100, accuracy: 0.84262294, det loss: 0.37886932, bbox loss: 0.02350236, all_loss: 0.3906205, lr:0.01 
2022-11-30 05:02:06.657937 : Epoch: 1, Step: 200, accuracy: 0.8032258, det loss: 0.39875543, bbox loss: 0.019412298, all_loss: 0.40846157, lr:0.01 
2022-11-30 05:02:33.731505 : Epoch: 1, Step: 300, accuracy: 0.9245283, det loss: 0.19905059, bbox loss: 0.018990787, all_loss: 0.20854598, lr:0.01 
2022-11-30 05:03:00.910741 : Epoch: 1, Step: 400, accuracy: 0.9006211, det loss: 0.27195117, bbox loss: 0.017941553, all_loss: 0.28092194, lr:0.01 
2022-11-30 05:03:29.320623 : Epoch: 1, Step: 500, accuracy: 0.9174041, det loss: 0.238488, bbox loss: 0.019246047, all_loss: 0.24811102, lr:0.01 
2022-11-30 05:03:57.843521 : Epoch: 1, Step: 600, accuracy: 0.875, det loss: 0.27276772, bbox loss: 0.0191084, all_loss

2022-11-30 05:26:57.330187 : Epoch: 1, Step: 5600, accuracy: 0.9520767, det loss: 0.15977448, bbox loss: 0.021659551, all_loss: 0.17060426, lr:0.01 
2022-11-30 05:27:24.945710 : Epoch: 1, Step: 5700, accuracy: 0.90849674, det loss: 0.2151803, bbox loss: 0.01941054, all_loss: 0.22488557, lr:0.01 
2022-11-30 05:27:52.517682 : Epoch: 1, Step: 5800, accuracy: 0.88922155, det loss: 0.2586762, bbox loss: 0.016918333, all_loss: 0.26713538, lr:0.01 
2022-11-30 05:28:20.109844 : Epoch: 1, Step: 5900, accuracy: 0.92789966, det loss: 0.20022689, bbox loss: 0.020894805, all_loss: 0.21067429, lr:0.01 
2022-11-30 05:28:47.564945 : Epoch: 1, Step: 6000, accuracy: 0.9292035, det loss: 0.18245155, bbox loss: 0.018812064, all_loss: 0.19185758, lr:0.01 
2022-11-30 05:29:15.186822 : Epoch: 1, Step: 6100, accuracy: 0.9213115, det loss: 0.21914308, bbox loss: 0.020899773, all_loss: 0.22959296, lr:0.01 
2022-11-30 05:29:42.712987 : Epoch: 1, Step: 6200, accuracy: 0.9245283, det loss: 0.15680349, bbox loss: 0

2022-11-30 05:52:27.951308 : Epoch: 2, Step: 3500, accuracy: 0.9419355, det loss: 0.14100802, bbox loss: 0.023225049, all_loss: 0.15262054, lr:0.01 
2022-11-30 05:52:55.582870 : Epoch: 2, Step: 3600, accuracy: 0.9184953, det loss: 0.19870913, bbox loss: 0.018323097, all_loss: 0.20787068, lr:0.01 
2022-11-30 05:53:23.062510 : Epoch: 2, Step: 3700, accuracy: 0.9016949, det loss: 0.20979013, bbox loss: 0.018698044, all_loss: 0.21913914, lr:0.01 
2022-11-30 05:53:50.639421 : Epoch: 2, Step: 3800, accuracy: 0.9144737, det loss: 0.21715568, bbox loss: 0.027679907, all_loss: 0.23099564, lr:0.01 
2022-11-30 05:54:18.221601 : Epoch: 2, Step: 3900, accuracy: 0.9213836, det loss: 0.23115197, bbox loss: 0.021890122, all_loss: 0.24209704, lr:0.01 
2022-11-30 05:54:45.822401 : Epoch: 2, Step: 4000, accuracy: 0.91925466, det loss: 0.1856885, bbox loss: 0.019383406, all_loss: 0.1953802, lr:0.01 
2022-11-30 05:55:13.525663 : Epoch: 2, Step: 4100, accuracy: 0.9240924, det loss: 0.19836906, bbox loss: 0.

2022-11-30 06:18:01.952520 : Epoch: 3, Step: 1400, accuracy: 0.9352751, det loss: 0.13906856, bbox loss: 0.020092, all_loss: 0.14911456, lr:0.01 
2022-11-30 06:18:31.662939 : Epoch: 3, Step: 1500, accuracy: 0.93290734, det loss: 0.17601109, bbox loss: 0.02015123, all_loss: 0.1860867, lr:0.01 
2022-11-30 06:18:59.152334 : Epoch: 3, Step: 1600, accuracy: 0.9495268, det loss: 0.13630916, bbox loss: 0.021834115, all_loss: 0.14722621, lr:0.01 
2022-11-30 06:19:26.789821 : Epoch: 3, Step: 1700, accuracy: 0.9163987, det loss: 0.17219505, bbox loss: 0.021897499, all_loss: 0.1831438, lr:0.01 
2022-11-30 06:19:54.670756 : Epoch: 3, Step: 1800, accuracy: 0.9189189, det loss: 0.1846334, bbox loss: 0.021078022, all_loss: 0.19517241, lr:0.01 
2022-11-30 06:20:22.217639 : Epoch: 3, Step: 1900, accuracy: 0.9315961, det loss: 0.1818385, bbox loss: 0.017728737, all_loss: 0.19070287, lr:0.01 
2022-11-30 06:20:49.667783 : Epoch: 3, Step: 2000, accuracy: 0.92749244, det loss: 0.20471433, bbox loss: 0.01817

2022-11-30 06:43:57.091397 : Epoch: 3, Step: 7000, accuracy: 0.95394737, det loss: 0.15599531, bbox loss: 0.018148018, all_loss: 0.16506931, lr:0.01 
2022-11-30 06:44:24.588427 : Epoch: 3, Step: 7100, accuracy: 0.9296188, det loss: 0.22540149, bbox loss: 0.016923493, all_loss: 0.23386323, lr:0.01 
2022-11-30 06:44:52.133052 : Epoch: 3, Step: 7200, accuracy: 0.9174603, det loss: 0.16909143, bbox loss: 0.01914221, all_loss: 0.17866254, lr:0.01 
2022-11-30 06:45:19.750293 : Epoch: 3, Step: 7300, accuracy: 0.9588235, det loss: 0.14813653, bbox loss: 0.01884682, all_loss: 0.15755993, lr:0.01 
2022-11-30 06:45:47.284615 : Epoch: 3, Step: 7400, accuracy: 0.9220779, det loss: 0.20249286, bbox loss: 0.019061167, all_loss: 0.21202345, lr:0.01 
2022-11-30 06:46:14.837350 : Epoch: 3, Step: 7500, accuracy: 0.9402985, det loss: 0.16504188, bbox loss: 0.024333239, all_loss: 0.1772085, lr:0.01 
2022-11-30 06:46:43.649890 : Epoch: 3, Step: 7600, accuracy: 0.9226006, det loss: 0.19407804, bbox loss: 0.0

...

2022-11-30 09:26:58.031204 : Epoch: 7, Step: 4200, accuracy: 0.93209875, det loss: 0.16950402, bbox loss: 0.018838918, all_loss: 0.17892347, lr:0.01 
2022-11-30 09:27:24.177816 : Epoch: 7, Step: 4300, accuracy: 0.9409938, det loss: 0.1378863, bbox loss: 0.022843577, all_loss: 0.14930809, lr:0.01 
2022-11-30 09:27:50.304042 : Epoch: 7, Step: 4400, accuracy: 0.9223881, det loss: 0.20894007, bbox loss: 0.020279568, all_loss: 0.21907985, lr:0.01 
2022-11-30 09:28:16.573750 : Epoch: 7, Step: 4500, accuracy: 0.94603175, det loss: 0.13088581, bbox loss: 0.02173634, all_loss: 0.14175399, lr:0.01 
2022-11-30 09:28:42.776781 : Epoch: 7, Step: 4600, accuracy: 0.9255663, det loss: 0.15833771, bbox loss: 0.025480287, all_loss: 0.17107786, lr:0.01 
2022-11-30 09:29:09.032114 : Epoch: 7, Step: 4700, accuracy: 0.94736844, det loss: 0.16199356, bbox loss: 0.029777788, all_loss: 0.17688246, lr:0.01 
2022-11-30 09:29:35.352819 : Epoch: 7, Step: 4800, accuracy: 0.9335443, det loss: 0.15879077, bbox loss: 

2022-11-30 09:51:28.282624 : Epoch: 8, Step: 2100, accuracy: 0.9315961, det loss: 0.1792493, bbox loss: 0.017891029, all_loss: 0.18819481, lr:0.01 
2022-11-30 09:51:54.454079 : Epoch: 8, Step: 2200, accuracy: 0.91875, det loss: 0.19404624, bbox loss: 0.020619642, all_loss: 0.20435606, lr:0.01 
2022-11-30 09:52:20.671971 : Epoch: 8, Step: 2300, accuracy: 0.9373041, det loss: 0.13728608, bbox loss: 0.02096752, all_loss: 0.14776984, lr:0.01 
2022-11-30 09:52:47.197219 : Epoch: 8, Step: 2400, accuracy: 0.9079365, det loss: 0.1972633, bbox loss: 0.01864088, all_loss: 0.20658374, lr:0.01 
2022-11-30 09:53:13.493424 : Epoch: 8, Step: 2500, accuracy: 0.9413681, det loss: 0.15565756, bbox loss: 0.022662098, all_loss: 0.16698861, lr:0.01 
2022-11-30 09:53:39.709718 : Epoch: 8, Step: 2600, accuracy: 0.91925466, det loss: 0.21727537, bbox loss: 0.02016498, all_loss: 0.22735785, lr:0.01 
2022-11-30 09:54:05.931126 : Epoch: 8, Step: 2700, accuracy: 0.9398734, det loss: 0.1627667, bbox loss: 0.021089

2022-11-30 10:15:48.721706 : Epoch: 9, Step: 0, accuracy: 0.9174603, det loss: 0.18285893, bbox loss: 0.018627297, all_loss: 0.19217257, lr:0.01 
2022-11-30 10:16:13.631895 : Epoch: 9, Step: 100, accuracy: 0.9354839, det loss: 0.1547044, bbox loss: 0.020491155, all_loss: 0.16494998, lr:0.01 
2022-11-30 10:16:38.885266 : Epoch: 9, Step: 200, accuracy: 0.9310345, det loss: 0.14478907, bbox loss: 0.018960888, all_loss: 0.15426952, lr:0.01 
2022-11-30 10:17:04.381171 : Epoch: 9, Step: 300, accuracy: 0.9361022, det loss: 0.18982846, bbox loss: 0.01925677, all_loss: 0.19945684, lr:0.01 
2022-11-30 10:17:30.042557 : Epoch: 9, Step: 400, accuracy: 0.94212216, det loss: 0.1463064, bbox loss: 0.021924326, all_loss: 0.15726855, lr:0.01 
2022-11-30 10:17:56.071126 : Epoch: 9, Step: 500, accuracy: 0.9463722, det loss: 0.16962714, bbox loss: 0.021903312, all_loss: 0.1805788, lr:0.01 
2022-11-30 10:18:22.316531 : Epoch: 9, Step: 600, accuracy: 0.9169231, det loss: 0.16715692, bbox loss: 0.019062568, 

2022-11-30 10:40:19.526975 : Epoch: 9, Step: 5600, accuracy: 0.9580645, det loss: 0.11389022, bbox loss: 0.022661543, all_loss: 0.125221, lr:0.01 
2022-11-30 10:40:45.715277 : Epoch: 9, Step: 5700, accuracy: 0.88125, det loss: 0.26917568, bbox loss: 0.027938463, all_loss: 0.28314492, lr:0.01 
2022-11-30 10:41:11.903543 : Epoch: 9, Step: 5800, accuracy: 0.95950156, det loss: 0.123762615, bbox loss: 0.020562254, all_loss: 0.13404374, lr:0.01 
2022-11-30 10:41:38.081557 : Epoch: 9, Step: 5900, accuracy: 0.90822786, det loss: 0.24431346, bbox loss: 0.019209271, all_loss: 0.2539181, lr:0.01 
2022-11-30 10:42:04.262150 : Epoch: 9, Step: 6000, accuracy: 0.92651755, det loss: 0.16978519, bbox loss: 0.019510897, all_loss: 0.17954063, lr:0.01 
2022-11-30 10:42:30.670394 : Epoch: 9, Step: 6100, accuracy: 0.916388, det loss: 0.17352156, bbox loss: 0.019346163, all_loss: 0.18319464, lr:0.01 
2022-11-30 10:42:56.949249 : Epoch: 9, Step: 6200, accuracy: 0.9113924, det loss: 0.2108897, bbox loss: 0.02

2022-11-30 11:04:08.840210 : Epoch: 10, Step: 3400, accuracy: 0.9481707, det loss: 0.15547565, bbox loss: 0.018355783, all_loss: 0.16465354, lr:0.01 
2022-11-30 11:04:35.110047 : Epoch: 10, Step: 3500, accuracy: 0.9254658, det loss: 0.1941336, bbox loss: 0.01849188, all_loss: 0.20337954, lr:0.01 
2022-11-30 11:05:01.243185 : Epoch: 10, Step: 3600, accuracy: 0.93811077, det loss: 0.1451279, bbox loss: 0.019246148, all_loss: 0.15475099, lr:0.01 
2022-11-30 11:05:27.631133 : Epoch: 10, Step: 3700, accuracy: 0.9223881, det loss: 0.20164856, bbox loss: 0.016687285, all_loss: 0.2099922, lr:0.01 
2022-11-30 11:05:53.845742 : Epoch: 10, Step: 3800, accuracy: 0.9413681, det loss: 0.19040826, bbox loss: 0.021563284, all_loss: 0.2011899, lr:0.01 
2022-11-30 11:06:20.150497 : Epoch: 10, Step: 3900, accuracy: 0.9346405, det loss: 0.15677229, bbox loss: 0.028839666, all_loss: 0.17119212, lr:0.01 
2022-11-30 11:06:46.361837 : Epoch: 10, Step: 4000, accuracy: 0.9537037, det loss: 0.14496374, bbox loss

# TRAIN RNet

In [84]:
import os
import sys
sys.path.append(os.getcwd())
from mtcnn.implement.imagedb import ImageDB
def trainr_net(imagedb,gt_imdb, model_store_path,
                end_epoch=16, frequent=200, lr=0.01, batch_size=128, use_cuda=False):


    train_rnet(model_store_path=model_store_path, end_epoch=end_epoch, imdb=gt_imdb, batch_size=batch_size, frequent=frequent, base_lr=lr, use_cuda=use_cuda)


# Load data

In [80]:
annotation_file = "./anno_store/imglist_anno_24.txt"
imagedb = ImageDB(annotation_file)
print(imagedb.num_images)
gt_imdb = imagedb.load_imdb()
gt_imdb = imagedb.append_flipped_images(gt_imdb)

751754
append flipped images to imdb 751754


In [85]:
model_store_path = "./model_store"
end_epoch = 5
lr = 0.01
batch_size = 32

use_cuda = False
frequent = 200

trainr_net(imagedb,gt_imdb, model_store_path, end_epoch, frequent, lr, batch_size, use_cuda)

 Epoch: 1, Step: 0, accuracy: 0.9158778362744283, det loss: 0.8224964160971424
 Epoch: 1, Step: 200, accuracy: 0.9237567898255917, det loss: 0.10316399369773926
 Epoch: 1, Step: 400, accuracy: 0.9157235248429473, det loss: 0.10257247810506319
 Epoch: 1, Step: 600, accuracy: 0.922097857228448, det loss: 0.10254249634959428
 Epoch: 1, Step: 800, accuracy: 0.9408156699427693, det loss: 0.1046163432975888
 Epoch: 1, Step: 1000, accuracy: 0.9151361962278328, det loss: 0.10180800506746501
 Epoch: 1, Step: 1200, accuracy: 0.9457660996898833, det loss: 0.10534603567039917
 Epoch: 1, Step: 1400, accuracy: 0.9126838797158799, det loss: 0.10158433365554348
 Epoch: 1, Step: 1600, accuracy: 0.9255137798400068, det loss: 0.10290526705092759
 Epoch: 1, Step: 1800, accuracy: 0.9122885742464304, det loss: 0.6613370280246283
 Epoch: 1, Step: 2000, accuracy: 0.9339010682224737, det loss: 0.10466126027141877
 Epoch: 1, Step: 2200, accuracy: 0.9132449612531448, det loss: 0.1016841327511303
 Epoch: 1, Step:

 Epoch: 3, Step: 3600, accuracy: 0.9277319241459958, det loss: 0.10308132490511064
 Epoch: 3, Step: 3800, accuracy: 0.9377597406836936, det loss: 0.10419552674263262
 Epoch: 3, Step: 4000, accuracy: 0.9399784257619709, det loss: 0.10444204730688565
 Epoch: 3, Step: 4200, accuracy: 0.9178373632331437, det loss: 0.10198192924812707
 Epoch: 3, Step: 4400, accuracy: 0.9494781647829645, det loss: 0.10549757386477383
 Epoch: 3, Step: 4600, accuracy: 0.9108899285830435, det loss: 0.10120999206478261
 Epoch: 3, Step: 4800, accuracy: 0.9308862256471586, det loss: 0.10343180284968428
 Epoch: 3, Step: 5000, accuracy: 0.9137302009624957, det loss: 0.10152557788472175
 Epoch: 3, Step: 5200, accuracy: 0.9391851518429666, det loss: 0.10435390576032962
 Epoch: 3, Step: 5400, accuracy: 0.9263363388542415, det loss: 0.10292625987269349
 Epoch: 3, Step: 5600, accuracy: 0.9258302407606901, det loss: 0.10287002675118778
 Epoch: 3, Step: 5800, accuracy: 0.9268068048502855, det loss: 0.10297853387225395
 Epo

 Epoch: 5, Step: 7200, accuracy: 0.922142756037472, det loss: 0.10246030622638577
 Epoch: 5, Step: 7400, accuracy: 0.9382248853037432, det loss: 0.10424720947819369
 Epoch: 5, Step: 7600, accuracy: 0.9458047109305486, det loss: 0.10508941232561651
 Epoch: 5, Step: 7800, accuracy: 0.9166575042299266, det loss: 0.10185083380332519
 Epoch: 5, Step: 8000, accuracy: 0.9459540089988343, det loss: 0.10510600099987048


# TRAIN ONet

In [104]:
import os
import sys
sys.path.append(os.getcwd())
from mtcnn.implement.imagedb import ImageDB
def traino_net(imagedb,gt_imdb, model_store_path,
                end_epoch=16, frequent=200, lr=0.01, batch_size=128, use_cuda=False):

    train_onet(model_store_path=model_store_path, end_epoch=end_epoch, imdb=gt_imdb, batch_size=batch_size, frequent=frequent, base_lr=lr, use_cuda=use_cuda)


# LOAD Data

In [89]:
annotation_file = "./anno_store/imglist_anno_48.txt"
imagedb = ImageDB(annotation_file)
gt_imdb = imagedb.load_imdb()
gt_imdb = imagedb.append_flipped_images(gt_imdb)

append flipped images to imdb 364031


In [105]:
annotation_file = "./anno_store/imglist_anno_48.txt"
model_store_path = "./model_store"
end_epoch = 5
lr = 0.002
batch_size = 128

use_cuda = False
frequent = 100


traino_net(imagedb,gt_imdb, model_store_path,
            end_epoch, frequent, lr, batch_size, use_cuda)


False
2022-11-30 21:09:10.671741 : Epoch: 1, Step: 0, accuracy: 0.9050021102950533, det loss: 0.7203, bbox loss: 0.034761935, landmark loss: 0.18996952, all_loss: 0.88205147, lr:0.002 
2022-11-30 21:09:27.228899 : Epoch: 1, Step: 100, accuracy: 0.9051510253226008, det loss: 0.0011511942, bbox loss: 0.014171217, landmark loss: 0.010275256, all_loss: 0.02483657, lr:0.002 
2022-11-30 21:09:43.620359 : Epoch: 1, Step: 200, accuracy: 0.9052886854523704, det loss: 0.0005035567, bbox loss: 0.004362326, landmark loss: 0.0059652766, all_loss: 0.011968155, lr:0.002 
2022-11-30 21:10:00.299915 : Epoch: 1, Step: 300, accuracy: 0.9054391047862079, det loss: 0.00044719613, bbox loss: 0.006428136, landmark loss: 0.0046734083, all_loss: 0.011224751, lr:0.002 
2022-11-30 21:10:17.155480 : Epoch: 1, Step: 400, accuracy: 0.9055867845462213, det loss: 0.0007489617, bbox loss: 0.0045429775, landmark loss: 0.0046415105, all_loss: 0.0102872215, lr:0.002 
2022-11-30 21:10:34.327077 : Epoch: 1, Step: 500, accu

2022-11-30 21:22:47.401134 : Epoch: 1, Step: 4300, accuracy: 0.9114758496890679, det loss: 5.8173937e-05, bbox loss: 0.0020710595, landmark loss: 0.0013419429, all_loss: 0.0033020894, lr:0.002 
2022-11-30 21:23:08.639551 : Epoch: 1, Step: 4400, accuracy: 0.91161098622828, det loss: 3.4372217e-05, bbox loss: 0.0025330125, landmark loss: 0.0011539899, all_loss: 0.0032782904, lr:0.002 
2022-11-30 21:23:29.879978 : Epoch: 1, Step: 4500, accuracy: 0.91177876157578, det loss: 0.00058105966, bbox loss: 0.0031871526, landmark loss: 0.0020542936, all_loss: 0.00545858, lr:0.002 
2022-11-30 21:23:51.258109 : Epoch: 1, Step: 4600, accuracy: 0.9119357273163364, det loss: 7.997367e-05, bbox loss: 0.0021746082, landmark loss: 0.0016921035, all_loss: 0.003906899, lr:0.002 
2022-11-30 21:24:12.362858 : Epoch: 1, Step: 4700, accuracy: 0.9120731251017364, det loss: 5.0199433e-05, bbox loss: 0.008499984, landmark loss: 0.000991446, all_loss: 0.006627319, lr:0.002 
2022-11-30 21:24:33.593038 : Epoch: 1, St

2022-11-30 21:38:18.248851 : Epoch: 2, Step: 2900, accuracy: 0.9179911099146267, det loss: 8.833052e-05, bbox loss: 0.0023558529, landmark loss: 0.0012618324, all_loss: 0.003376925, lr:0.002 
2022-11-30 21:38:41.779142 : Epoch: 2, Step: 3000, accuracy: 0.9181409591216825, det loss: 7.965628e-05, bbox loss: 0.0028869347, landmark loss: 0.0013974279, all_loss: 0.0038920278, lr:0.002 
2022-11-30 21:39:04.911394 : Epoch: 2, Step: 3100, accuracy: 0.9182651826843522, det loss: 2.91917e-05, bbox loss: 0.0017670707, landmark loss: 0.0009932492, all_loss: 0.0025734697, lr:0.002 
2022-11-30 21:39:28.938733 : Epoch: 2, Step: 3200, accuracy: 0.9183994000542606, det loss: 4.066956e-05, bbox loss: 0.0027485574, landmark loss: 0.0013020885, all_loss: 0.0036348028, lr:0.002 
2022-11-30 21:39:52.444034 : Epoch: 2, Step: 3300, accuracy: 0.9185449850856777, det loss: 3.581472e-05, bbox loss: 0.004821332, landmark loss: 0.0012740077, all_loss: 0.0048324624, lr:0.002 
2022-11-30 21:40:15.588523 : Epoch: 2,

2022-11-30 21:54:46.098834 : Epoch: 3, Step: 1500, accuracy: 0.9244111437870339, det loss: 3.1986023e-05, bbox loss: 0.0026309392, landmark loss: 0.0013323005, all_loss: 0.003602603, lr:0.002 
2022-11-30 21:55:06.057244 : Epoch: 3, Step: 1600, accuracy: 0.9245536771169796, det loss: 7.290578e-05, bbox loss: 0.0017749658, landmark loss: 0.0011294028, all_loss: 0.0028174082, lr:0.002 
2022-11-30 21:55:27.216957 : Epoch: 3, Step: 1700, accuracy: 0.9247081217903615, det loss: 5.3565345e-05, bbox loss: 0.0011490746, landmark loss: 0.001000707, all_loss: 0.0022333576, lr:0.002 
2022-11-30 21:55:48.739832 : Epoch: 3, Step: 1800, accuracy: 0.9248326523175914, det loss: 3.1917312e-05, bbox loss: 0.002592131, landmark loss: 0.0009878471, all_loss: 0.003062583, lr:0.002 
2022-11-30 21:56:10.073992 : Epoch: 3, Step: 1900, accuracy: 0.92498436491833, det loss: 3.599696e-05, bbox loss: 0.0022981314, landmark loss: 0.0008459698, all_loss: 0.002676631, lr:0.002 
2022-11-30 21:56:31.006994 : Epoch: 3, 

2022-11-30 22:10:04.703162 : Epoch: 4, Step: 100, accuracy: 0.9307053164663277, det loss: 8.291626e-05, bbox loss: 0.011985786, landmark loss: 0.009389623, all_loss: 0.02134224, lr:0.002 
2022-11-30 22:10:27.317482 : Epoch: 4, Step: 200, accuracy: 0.9308403868379299, det loss: 0.00024411571, bbox loss: 0.008666601, landmark loss: 0.007759954, all_loss: 0.017035184, lr:0.002 
2022-11-30 22:10:50.083845 : Epoch: 4, Step: 300, accuracy: 0.9310036157256268, det loss: 6.6401575e-05, bbox loss: 0.007586976, landmark loss: 0.007799812, all_loss: 0.016305026, lr:0.002 
2022-11-30 22:11:12.779360 : Epoch: 4, Step: 400, accuracy: 0.9311612695477385, det loss: 7.984799e-05, bbox loss: 0.005305514, landmark loss: 0.00746496, all_loss: 0.014444628, lr:0.002 
2022-11-30 22:11:35.417819 : Epoch: 4, Step: 500, accuracy: 0.9313229672494782, det loss: 3.2137996e-05, bbox loss: 0.008459319, landmark loss: 0.010484273, all_loss: 0.020827712, lr:0.002 
2022-11-30 22:11:59.390309 : Epoch: 4, Step: 600, accu

2022-11-30 22:26:51.833825 : Epoch: 4, Step: 4400, accuracy: 0.9372057858246167, det loss: 0.00041285186, bbox loss: 0.006756978, landmark loss: 0.0073945033, all_loss: 0.015476223, lr:0.002 
2022-11-30 22:27:15.387808 : Epoch: 4, Step: 4500, accuracy: 0.9373536172683938, det loss: 0.00011200661, bbox loss: 0.0059544756, landmark loss: 0.007539833, all_loss: 0.01497204, lr:0.002 
2022-11-30 22:27:39.202325 : Epoch: 4, Step: 4600, accuracy: 0.9374987739640571, det loss: 2.2445443e-05, bbox loss: 0.005590824, landmark loss: 0.0068493667, all_loss: 0.0136465, lr:0.002 
2022-11-30 22:28:02.461464 : Epoch: 4, Step: 4700, accuracy: 0.9376600549520572, det loss: 6.393477e-05, bbox loss: 0.0052263304, landmark loss: 0.005285691, all_loss: 0.011115482, lr:0.002 
2022-11-30 22:28:25.786664 : Epoch: 4, Step: 4800, accuracy: 0.9377961209961462, det loss: 7.9380385e-05, bbox loss: 0.0061273025, landmark loss: 0.003976481, all_loss: 0.009704608, lr:0.002 
2022-11-30 22:28:49.225225 : Epoch: 4, Step:

2022-11-30 22:45:00.062756 : Epoch: 5, Step: 3000, accuracy: 0.9434857800767592, det loss: 0.00013304599, bbox loss: 0.005343939, landmark loss: 0.0048946883, all_loss: 0.010654833, lr:0.002 
2022-11-30 22:45:25.118366 : Epoch: 5, Step: 3100, accuracy: 0.9436428465030775, det loss: 0.00012911286, bbox loss: 0.006220228, landmark loss: 0.0043610143, all_loss: 0.010376949, lr:0.002 
2022-11-30 22:45:50.633441 : Epoch: 5, Step: 3200, accuracy: 0.9438053384570887, det loss: 0.0002118813, bbox loss: 0.0047525545, landmark loss: 0.0039220895, all_loss: 0.008904172, lr:0.002 
2022-11-30 22:46:14.855237 : Epoch: 5, Step: 3300, accuracy: 0.9439341053686013, det loss: 8.8270375e-05, bbox loss: 0.0056676087, landmark loss: 0.0035995215, all_loss: 0.008870464, lr:0.002 
2022-11-30 22:46:39.357698 : Epoch: 5, Step: 3400, accuracy: 0.9440688463324179, det loss: 7.610773e-05, bbox loss: 0.005969473, landmark loss: 0.004840912, all_loss: 0.010903938, lr:0.002 
2022-11-30 22:47:03.576846 : Epoch: 5, St

# Test set

In [111]:
import cv2
from mtcnn.implement.detect import create_mtcnn_net, MtcnnDetector
pnet, rnet, onet = create_mtcnn_net(p_model_path="./final_model/pnet_epoch.pt", r_model_path="./final_model/rnet_epoch.pt", o_model_path="./final_model/onet_epoch.pt", use_cuda=False)
mtcnn_detector = MtcnnDetector(pnet=pnet, rnet=rnet, onet=onet, min_face_size=24)

In [108]:
list_image_path='./data_set/face_landmark/CNN_FacePoint/train/testImageList.txt'

In [157]:
def get_file_path(path):
    new_path=path.split('\\')
    parent_path='./data_set/face_landmark/CNN_FacePoint/train/'
    parent_path = parent_path+new_path[0]+'/'+new_path[1]
    return parent_path

In [173]:
def run_test(list_image_path,mtcnn_detector):
    num_pic=0
    sum_accuracy=0
    with open(list_image_path) as f:
        lines = f.readlines()
        for line in lines:
            data = line.split(" ")
            file_path = get_file_path(data[0])
            oboxs = np.array([int(data[1]),int(data[3]),int(data[2]),int(data[4])])
            img = cv2.imread(file_path)
            acc = get_image_box_accuracy(img,oboxs,mtcnn_detector)
            print("Picture "+str(num_pic+1)+ " accuracy is",acc)
            num_pic+=1
            sum_accuracy+=acc
    print("Average accuracy is",sum_accuracy/num_pic)
run_test(list_image_path,mtcnn_detector)

time cost 0.062  pnet 0.037  rnet 0.012  onet 0.014
Picture 1 accuracy is 0.960070741945096
time cost 0.020  pnet 0.011  rnet 0.005  onet 0.003
Picture 2 accuracy is 0.9974499364469506
time cost 0.021  pnet 0.010  rnet 0.006  onet 0.005
Picture 3 accuracy is 0.9905278987481818
time cost 0.038  pnet 0.013  rnet 0.014  onet 0.011
Picture 4 accuracy is 0.9816810249147488
time cost 0.019  pnet 0.010  rnet 0.007  onet 0.002
Picture 5 accuracy is 0.9634690689831618
time cost 0.031  pnet 0.012  rnet 0.012  onet 0.006
Picture 6 accuracy is 0.9968480075275734
time cost 0.028  pnet 0.011  rnet 0.009  onet 0.008
Picture 7 accuracy is 0.9719752433942233
time cost 0.026  pnet 0.012  rnet 0.010  onet 0.004
Picture 8 accuracy is 0.9909078665360058
time cost 0.027  pnet 0.013  rnet 0.009  onet 0.005
Picture 9 accuracy is 0.9878372440389825
time cost 0.035  pnet 0.014  rnet 0.014  onet 0.007
Picture 10 accuracy is 0.9455520731389386
time cost 0.023  pnet 0.011  rnet 0.007  onet 0.005
Picture 11 accurac

time cost 0.030  pnet 0.014  rnet 0.014  onet 0.003
Picture 92 accuracy is 0.9901555825101733
time cost 0.041  pnet 0.018  rnet 0.016  onet 0.007
Picture 93 accuracy is 0.980371810047888
time cost 0.019  pnet 0.010  rnet 0.006  onet 0.003
Picture 94 accuracy is 0.9650871361014133
time cost 0.026  pnet 0.012  rnet 0.007  onet 0.007
Picture 95 accuracy is 0.9901107445605172
time cost 0.032  pnet 0.014  rnet 0.010  onet 0.008
Picture 96 accuracy is 0.9471121307274168
time cost 0.024  pnet 0.012  rnet 0.006  onet 0.007
Picture 97 accuracy is 0.9434055354757971
time cost 0.024  pnet 0.012  rnet 0.008  onet 0.004
Picture 98 accuracy is 0.9198972761424762
time cost 0.036  pnet 0.014  rnet 0.015  onet 0.007
Picture 99 accuracy is 0.9798955981598234
time cost 0.027  pnet 0.014  rnet 0.008  onet 0.005
Picture 100 accuracy is 0.9511984489655102
time cost 0.035  pnet 0.014  rnet 0.016  onet 0.006
Picture 101 accuracy is 0.9577854792363761
time cost 0.020  pnet 0.009  rnet 0.005  onet 0.005
Picture

time cost 0.037  pnet 0.016  rnet 0.015  onet 0.006
Picture 179 accuracy is 0.954450558037186
time cost 0.023  pnet 0.011  rnet 0.007  onet 0.005
Picture 180 accuracy is 0.9801729588287927
time cost 0.030  pnet 0.013  rnet 0.011  onet 0.007
Picture 181 accuracy is 0.9462709941999164
time cost 0.031  pnet 0.015  rnet 0.014  onet 0.002
Picture 182 accuracy is 0.9748737970979763
time cost 0.039  pnet 0.016  rnet 0.020  onet 0.003
Picture 183 accuracy is 0.9306005611609539
time cost 0.026  pnet 0.012  rnet 0.009  onet 0.005
Picture 184 accuracy is 0.9885138197299568
time cost 0.036  pnet 0.014  rnet 0.013  onet 0.009
Picture 185 accuracy is 0.9497279042661577
time cost 0.034  pnet 0.013  rnet 0.013  onet 0.008
Picture 186 accuracy is 0.9815411768984642
time cost 0.024  pnet 0.012  rnet 0.008  onet 0.004
Picture 187 accuracy is 0.9739953039472856
time cost 0.029  pnet 0.013  rnet 0.012  onet 0.004
Picture 188 accuracy is 0.9676177887414089
time cost 0.029  pnet 0.014  rnet 0.012  onet 0.003

time cost 0.032  pnet 0.015  rnet 0.011  onet 0.006
Picture 273 accuracy is 0.9763972993962978
time cost 0.031  pnet 0.013  rnet 0.010  onet 0.007
Picture 274 accuracy is 0.9908682917261062
time cost 0.035  pnet 0.015  rnet 0.015  onet 0.005
Picture 275 accuracy is 0.9390061768468037
time cost 0.022  pnet 0.011  rnet 0.006  onet 0.004
Picture 276 accuracy is 0.9291180762968192
time cost 0.042  pnet 0.016  rnet 0.014  onet 0.012
Picture 277 accuracy is 0.9532728051646993
time cost 0.034  pnet 0.012  rnet 0.013  onet 0.008
Picture 278 accuracy is 0.939985675014957
time cost 0.024  pnet 0.009  rnet 0.007  onet 0.007
Picture 279 accuracy is 0.9539713704986271
time cost 0.027  pnet 0.013  rnet 0.008  onet 0.006
Picture 280 accuracy is 0.9973557545760254
time cost 0.023  pnet 0.011  rnet 0.006  onet 0.006
Picture 281 accuracy is 0.982587286316151
time cost 0.020  pnet 0.009  rnet 0.005  onet 0.005
Picture 282 accuracy is 0.9911183246103029
time cost 0.027  pnet 0.012  rnet 0.009  onet 0.006


time cost 0.020  pnet 0.010  rnet 0.005  onet 0.005
Picture 362 accuracy is 0.994270976950943
time cost 0.028  pnet 0.014  rnet 0.009  onet 0.005
Picture 363 accuracy is 0.9729757872771351
time cost 0.021  pnet 0.010  rnet 0.006  onet 0.005
Picture 364 accuracy is 0.9738317156808141
time cost 0.038  pnet 0.013  rnet 0.010  onet 0.015
Picture 365 accuracy is 0.9690845397657321
time cost 0.037  pnet 0.016  rnet 0.015  onet 0.007
Picture 366 accuracy is 0.9685453501076411
time cost 0.026  pnet 0.011  rnet 0.007  onet 0.008
Picture 367 accuracy is 0.6632035976546848
time cost 0.033  pnet 0.013  rnet 0.014  onet 0.005
Picture 368 accuracy is 0.9455206040951347
time cost 0.043  pnet 0.017  rnet 0.016  onet 0.010
Picture 369 accuracy is 0.8784370560854857
time cost 0.030  pnet 0.012  rnet 0.010  onet 0.008
Picture 370 accuracy is 0.9847944499955363
time cost 0.023  pnet 0.010  rnet 0.006  onet 0.007
Picture 371 accuracy is 0.9741948409868937
Picture 372 accuracy is 0
time cost 0.031  pnet 0.0

...

time cost 0.020  pnet 0.009  rnet 0.007  onet 0.004
Picture 3072 accuracy is 0.9036102879741651
time cost 0.023  pnet 0.009  rnet 0.007  onet 0.007
Picture 3073 accuracy is 0.9710945057641054
time cost 0.020  pnet 0.008  rnet 0.007  onet 0.006
Picture 3074 accuracy is 0.9739869828541879
Picture 3075 accuracy is 0
time cost 0.025  pnet 0.011  rnet 0.011  onet 0.003
Picture 3076 accuracy is 0.9952258903404382
time cost 0.017  pnet 0.007  rnet 0.007  onet 0.004
Picture 3077 accuracy is 0.9464371684796207
time cost 0.021  pnet 0.008  rnet 0.008  onet 0.005
Picture 3078 accuracy is 0.9414057581461023
time cost 0.022  pnet 0.007  rnet 0.007  onet 0.008
Picture 3079 accuracy is 0.9580424304459416
time cost 0.027  pnet 0.010  rnet 0.010  onet 0.007
Picture 3080 accuracy is 0.9673681266712677
time cost 0.031  pnet 0.010  rnet 0.012  onet 0.009
Picture 3081 accuracy is 0.9500532410471291
time cost 0.022  pnet 0.010  rnet 0.006  onet 0.006
Picture 3082 accuracy is 0.9354082085909032
time cost 0.0

time cost 0.026  pnet 0.009  rnet 0.010  onet 0.008
Picture 3167 accuracy is 0.9328827715832778
time cost 0.020  pnet 0.007  rnet 0.007  onet 0.005
Picture 3168 accuracy is 0.9591013054896272
time cost 0.021  pnet 0.010  rnet 0.006  onet 0.005
Picture 3169 accuracy is 0.9224462825865101
time cost 0.023  pnet 0.009  rnet 0.008  onet 0.006
Picture 3170 accuracy is 0.900490975831469
time cost 0.023  pnet 0.009  rnet 0.009  onet 0.005
Picture 3171 accuracy is 0.9809856549970729
time cost 0.028  pnet 0.008  rnet 0.011  onet 0.009
Picture 3172 accuracy is 0.9358401991631929
time cost 0.023  pnet 0.007  rnet 0.008  onet 0.008
Picture 3173 accuracy is 0.975151274757696
time cost 0.021  pnet 0.008  rnet 0.007  onet 0.006
Picture 3174 accuracy is 0.928152735097661
time cost 0.020  pnet 0.008  rnet 0.007  onet 0.004
Picture 3175 accuracy is 0.8926462712776683
time cost 0.022  pnet 0.007  rnet 0.007  onet 0.008
Picture 3176 accuracy is 0.9369500246003916
time cost 0.020  pnet 0.008  rnet 0.007  on

time cost 0.019  pnet 0.010  rnet 0.004  onet 0.005
Picture 3259 accuracy is 0.9824078058410604
Picture 3260 accuracy is 0
time cost 0.026  pnet 0.010  rnet 0.008  onet 0.008
Picture 3261 accuracy is 0.9821780251233101
time cost 0.023  pnet 0.011  rnet 0.008  onet 0.003
Picture 3262 accuracy is 0.9986784592624335
time cost 0.019  pnet 0.007  rnet 0.007  onet 0.005
Picture 3263 accuracy is 0.9303490237767875
Picture 3264 accuracy is 0
Picture 3265 accuracy is 0
Picture 3266 accuracy is 0
time cost 0.021  pnet 0.009  rnet 0.006  onet 0.006
Picture 3267 accuracy is 0.9845821720087241
time cost 0.025  pnet 0.009  rnet 0.007  onet 0.009
Picture 3268 accuracy is 0.9975272016434658
time cost 0.029  pnet 0.009  rnet 0.011  onet 0.008
Picture 3269 accuracy is 0.9716099313995132
time cost 0.027  pnet 0.011  rnet 0.010  onet 0.006
Picture 3270 accuracy is 0.7752400642835953
time cost 0.017  pnet 0.006  rnet 0.004  onet 0.006
Picture 3271 accuracy is 0.9478795228916908
time cost 0.029  pnet 0.011 

time cost 0.022  pnet 0.008  rnet 0.006  onet 0.009
Picture 3354 accuracy is 0.9868962035656478
time cost 0.021  pnet 0.009  rnet 0.008  onet 0.004
Picture 3355 accuracy is 0.989205825473413
time cost 0.027  pnet 0.010  rnet 0.010  onet 0.007
Picture 3356 accuracy is 0.9592244457810528
time cost 0.021  pnet 0.007  rnet 0.007  onet 0.006
Picture 3357 accuracy is 0.9976655157598506
time cost 0.020  pnet 0.008  rnet 0.006  onet 0.005
Picture 3358 accuracy is 0.9655879313558361
time cost 0.026  pnet 0.010  rnet 0.010  onet 0.006
Picture 3359 accuracy is 0.9504386258882737
time cost 0.021  pnet 0.007  rnet 0.009  onet 0.005
Picture 3360 accuracy is 0.9923993753369225
time cost 0.028  pnet 0.010  rnet 0.009  onet 0.008
Picture 3361 accuracy is 0.9805683429393268
time cost 0.025  pnet 0.009  rnet 0.010  onet 0.006
Picture 3362 accuracy is 0.9526547678045286
time cost 0.021  pnet 0.008  rnet 0.007  onet 0.005
Picture 3363 accuracy is 0.9396381995843408
time cost 0.022  pnet 0.008  rnet 0.007  

time cost 0.025  pnet 0.009  rnet 0.010  onet 0.006
Picture 3446 accuracy is 0.9340871897091892
time cost 0.025  pnet 0.010  rnet 0.008  onet 0.008
Picture 3447 accuracy is 0.985347304818983
time cost 0.024  pnet 0.009  rnet 0.009  onet 0.006
Picture 3448 accuracy is 0.8738602422637832
time cost 0.010  pnet 0.005  rnet 0.002  onet 0.002
Picture 3449 accuracy is 0.9709477550305682
time cost 0.021  pnet 0.009  rnet 0.005  onet 0.007
Picture 3450 accuracy is 0.9225917804055942
time cost 0.028  pnet 0.010  rnet 0.012  onet 0.006
Picture 3451 accuracy is 0.9707230989880256
time cost 0.025  pnet 0.009  rnet 0.010  onet 0.006
Picture 3452 accuracy is 0.9963247150575331
Picture 3453 accuracy is 0
time cost 0.021  pnet 0.009  rnet 0.007  onet 0.005
Picture 3454 accuracy is 0.9688379389154353
time cost 0.032  pnet 0.011  rnet 0.015  onet 0.006
Picture 3455 accuracy is 0.8866169693254915
Picture 3456 accuracy is 0
time cost 0.027  pnet 0.011  rnet 0.012  onet 0.003
Picture 3457 accuracy is 0.9354